In [6]:
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

QUESTIONS_PROMPT = """
You are a powerful submind that helps people, companies, and organizations write manifestos that draw people into their orbit online.

You are looking at a web page, and trying to answer these questions:

What core problem or issue are you trying to address?
What is your personal story or journey that led you to this point?
What are your fundamental values and principles?
What is your vision for the future?
Who is your target audience?
What specific actions or changes do you want to inspire?
What makes your perspective or approach unique?
How do you want to emotionally connect with your audience?
What are the key obstacles or challenges you face?
How do you define success for your manifesto?

Here are the answers you have so far: {answers}

Here is the next page: {page}

Based on the page, update your answers to be as full and complete as possible based on the new input.
"""

EVALUATION_PROMPT = """
Given a customized manifesto, score the manifesto on these categories, rating each 1-10.
Clarity of purpose: Is the main goal or problem clearly articulated?
Authenticity: Does it reflect the true voice and values of the entity?
Emotional appeal: Does it connect on an emotional level with its intended audience?
Call to action: Does it provide clear, actionable steps or inspire specific changes?
Relevance: Does it address current and pertinent issues for its audience?
Uniqueness: Does it offer a fresh perspective or approach?
Consistency: Are the ideas and tone consistent throughout the manifesto?
Structure: Is it organized in a way that effectively communicates its message?
Vision: Does it paint a compelling picture of the desired future?
Alignment: Do the proposed solutions align with the stated values and principles?

"""

In [21]:
from decouple import config
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone  import PineconeVectorStore
from pinecone import Pinecone
import uuid 
from langchain_experimental.text_splitter import SemanticChunker
from langchain_text_splitters import RecursiveCharacterTextSplitter
import json
from urllib.parse import urlparse, urljoin
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import re
from langchain.document_loaders import BSHTMLLoader
from langchain_chroma import Chroma
import requests
from langchain_anthropic import ChatAnthropic

import chromadb

persistent_client = chromadb.PersistentClient()
collection = persistent_client.get_or_create_collection("testing_manifestos")


embeddings = OpenAIEmbeddings(
                            model="text-embedding-3-small",
                            openai_api_key=config("OPENAI_API_KEY"),
                            openai_api_base=config('OPENAI_API_BASE'),
                            headers={
                                "Helicone-Auth": f"Bearer {config('HELICONE_API_KEY')}"
                            })

langchain_chroma = Chroma(
    client=persistent_client,
    collection_name="testing_manifestos",
    embedding_function=embeddings,
)


model_claude = ChatAnthropic(model_name="claude-3-5-sonnet-20240620",
                                 anthropic_api_key=config("ANTHROPIC_API_KEY"),
                                 anthropic_api_url="https://anthropic.hconeai.com/",
                                 max_tokens=4096,
                                 model_kwargs={
                                     "extra_headers": {
                                         "Helicone-Auth": f"Bearer {config('HELICONE_API_KEY')}",
                                         "Helicone-Property-Step": "Manifesto Bot",
                                         "Helicone-Property-UUID": "Rob Hardy, Manifesto King"

                                     }
                                 }
                                 )
prompt = ChatPromptTemplate.from_template(QUESTIONS_PROMPT)
chain = prompt | model_claude | StrOutputParser()

    

webpages = {}
text_splitter = SemanticChunker(embeddings)
backup_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)
def add_page_to_vectorstore(webpage_text, webpage_name, webpage_url, namespace):
    chunks = text_splitter.split_text(webpage_text)
    metadatas = []
    ids = []
    chunks_to_save = []
    webpage_id = str(uuid.uuid4())
    webpages[namespace] = {
        f'{webpage_id}': {
            "title": webpage_name,
            "url": webpage_url
        }
    }
    for chunk in chunks:
        chunk_id = str(uuid.uuid4())
        metadata = {
            "webpage_id": webpage_id,
            "chunk_id": chunk_id,
            "webpage_name": webpage_name,
            "url": webpage_url,
            "text": chunk
        }

        # test metadata size
        size = sys.getsizeof(json.dumps(metadata))

        if size > 40960:
            # remove the snippet, because it's too big
            snippet.delete()
            # need to split into smaller chunks
            smaller_chunks = backup_text_splitter.split_text(chunk)
            for smaller_chunk in smaller_chunks:
                smaller_id = str(uuid.uuid4())
                
                smaller_metadata = {
                         "webpage_id": webpage_id,
                        "chunk_id": smaller_id,
                        "webpage_name": webpage_name,
                        "url": webpage_url,
                        "text": smaller_chunk
                }
                metadatas.append(smaller_metadata)
                ids.append(smaller_id)
                chunks_to_save.append(smaller_chunk)
            continue
        else:
            chunks_to_save.append(chunk)
            metadatas.append(metadata)
            ids.append(chunk_id)

    collection.add(documents=chunks_to_save, metadatas=metadatas, ids=ids)


links_to_process = []


def url_to_filename(url):
    safe_name = re.sub(r"[^\w\-_]", "_", url)
    return safe_name

def clean_string(input_string):
    cleaned_string = "".join(char for char in input_string.lower() if char.isalpha())
    return cleaned_string

def download_file(download_url, file_name):
    req = Request(download_url, headers={'User-Agent' : "Magic Browser"})

    response = urlopen(req)
    f = open(file_name, 'wb')
    f.write(response.read())
    f.close()

def process_page(soup: BeautifulSoup, url, namespace, known_so_far:str):
    # Placeholder function
    # Add your scraping code here
    title = soup.title.string if soup.title else url
    content = soup.get_text(strip=True)
    print(f"----- Processing page: {soup.title.string} -----")
    print(content)
    file_name = url_to_filename(url)
    download_file(url, file_name)
    loader = BSHTMLLoader(file_name)

    data = loader.load()

    content = soup.get_text(strip=True)

    response = chain.invoke(
        {"answers": known_so_far,
         "page": content,
         })
    print(response)

    add_page_to_vectorstore(content, title, url, namespace)
    return response
       


def is_valid(url: str) -> bool:
    parsed = urlparse(url)
    return bool(parsed.netloc) and bool(parsed.scheme)


def get_all_links(url: str, root_url: str):
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:15.0) Gecko/20100101 Firefox/15.0.1'}
    source_code = requests.get(url, headers=headers)
    soup = BeautifulSoup(source_code.content, 'html.parser')

    for link in soup.find_all('a'):
        link_href = link.get('href')
        if link_href:
            absolute_url = urljoin(root_url, link_href)
            if is_valid(absolute_url) and absolute_url.startswith(root_url):
                yield absolute_url

def crawl(base_url: str, namespace: str):
    print(f"----- Crawling: {base_url} -----")
    current_answers = ""
    crawled_urls = set()
    to_crawl = {base_url}

    session = requests.Session()
    session.verify = False
    while to_crawl:
        url = to_crawl.pop()
        try:
            print(f"----- Crawling: {url} -----")
            headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:15.0) Gecko/20100101 Firefox/15.0.1'}
            response = session.get(url, headers=headers)

            if response.status_code != 200:
                continue

            soup = BeautifulSoup(response.content, 'html.parser')
            current_answers = process_page(soup, url, namespace, current_answers)

            crawled_urls.add(url)

            for link in get_all_links(url, base_url):
                if link not in crawled_urls:
                    to_crawl.add(link)
        except Exception as e:
            print(f"Error crawling {url}: {e}")

In [13]:
crawl("https://foster.co", "nothingthatmatters")

----- Crawling: https://foster.co -----
----- Crawling: https://foster.co -----


/Users/leoguinan/Library/Caches/pypoetry/virtualenvs/backend-hNkkeHAR-py3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'foster.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/leoguinan/Library/Caches/pypoetry/virtualenvs/backend-hNkkeHAR-py3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.foster.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


----- Processing page: Foster — Experience the joy of writing for its own sake -----
Foster — Experience the joy of writing for its own sakeCohortsMembershipExperience the joy ofWriting for its Own SakeHi, we’re Foster. We’ve cultivated a way of writing together that leads to truer expressions of creativity and self. It’s fun, it’s interesting, and it makes writing easier, less lonely, and more fulfilling. Curious? Come try our signature Writing Circle and see what we’re all about.Join a Free Writing CircleFoster in the Words of the Collective“This writing circle has changed my life. Thank you for sharing this special gift with the world.”Maja T.“Thanks to Foster, I have reached an unimaginable level of acceptance and understanding of who I am. The community was critical in processing the loss of my mom, clarifying my vocation, and meeting people I love more than family.”Nicolas F.“Foster has helped me unlock myself in ways I didn’t know were possible.”Leo G."I can map the success in m

/Users/leoguinan/Library/Caches/pypoetry/virtualenvs/backend-hNkkeHAR-py3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'foster.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/leoguinan/Library/Caches/pypoetry/virtualenvs/backend-hNkkeHAR-py3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.foster.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Here are the updated answers to the questions based on the new information provided on the Foster website:

1. What core problem or issue are you trying to address?
Foster is trying to address the issue that writing has often become co-opted by the need for perfection or performance, rather than being an inherently joyful and transformative practice. They aim to help people rediscover the pleasure and benefits of writing for its own sake.

2. What is your personal story or journey that led you to this point?
The founders of Foster, including Dan Hunt, Sara Campbell, Minnow Park, Steph Soussloff, and Lyle McKeany, have all personally experienced the power of writing as a tool for self-discovery, creativity, and community. They want to share this gift with others and cultivate a supportive collective of writers.

3. What are your fundamental values and principles?
The core values and principles of Foster include: embracing the joy and process of writing, fostering an embodied, authentic 

...

/Users/leoguinan/Library/Caches/pypoetry/virtualenvs/backend-hNkkeHAR-py3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'foster.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/leoguinan/Library/Caches/pypoetry/virtualenvs/backend-hNkkeHAR-py3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.foster.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/leoguinan/Library/Caches/pypoetry/virtualenvs/backend-hNkkeHAR-py3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'foster.co'. Adding certificate verification 

----- Crawling: https://foster.co/signin?next=%2Fpublic-writing-circle -----
----- Crawling: https://foster.co/sn8s6irq -----


/Users/leoguinan/Library/Caches/pypoetry/virtualenvs/backend-hNkkeHAR-py3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'foster.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/leoguinan/Library/Caches/pypoetry/virtualenvs/backend-hNkkeHAR-py3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.foster.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/leoguinan/Library/Caches/pypoetry/virtualenvs/backend-hNkkeHAR-py3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'foster.co'. Adding certificate verification 

----- Crawling: https://foster.co/mfzi1dmd -----
----- Crawling: https://foster.co/discover -----


/Users/leoguinan/Library/Caches/pypoetry/virtualenvs/backend-hNkkeHAR-py3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.foster.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/leoguinan/Library/Caches/pypoetry/virtualenvs/backend-hNkkeHAR-py3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'foster.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/leoguinan/Library/Caches/pypoetry/virtualenvs/backend-hNkkeHAR-py3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.foster.co'. Adding certificate verificat

In [22]:
GUIDELINES_PROMPT = """
You are a powerful submind used to write manifestos.

The manifestos you write adhere to this outline nad guidelines:
I. Introduction
A. Personal narrative or compelling hook
B. Clear statement of purpose
II. Problem Identification
A. Detailed description of the issue
B. Emotional appeal to create urgency
C. Analysis of root causes
III. Vision and Values
A. Articulation of desired future state
B. Core principles and beliefs
C. Philosophical underpinnings
IV. Solutions and Strategies
A. Proposed actions to address the problem
B. Long-term goals and short-term steps
C. Explanation of key concepts or methods
V. Call to Action
A. Specific ways for readers to get involved
B. Emphasis on personal responsibility
C. Invitation to join a community or movement
VI. Potential Impact
A. Description of positive outcomes
B. How changes will benefit individuals and society
C. Vision for broader transformation
VII. Addressing Counterarguments
A. Anticipation of potential criticisms
B. Thoughtful responses to objections
VIII. Conclusion
A. Restatement of core message
B. Inspirational closing thoughts
C. Final call to action
Additional Elements to Consider:

Use of metaphors or analogies to explain complex ideas
Integration of relevant data or research
Personal anecdotes to add authenticity
Clear structure with subheadings for easy navigation
Emphasis on building trust and authenticity
Balance between critique and optimism
Acknowledgment of complexity while offering clear direction

Use the following questions and answers to write a manifesto:

1. What core problem or issue are you trying to address?
Foster is addressing the challenge that writing can often feel isolating and daunting, rather than a joyful, transformative practice. They aim to provide a supportive community and structure to help people rediscover the pleasure and benefits of regular, embodied writing.

2. What is your personal story or journey that led you to this point?
The Foster team, including facilitators like Lyle McKeany, Sara Campbell, Minnow Park, Stephanie Soussloff, Dan Hunt, and others, have all personally experienced the power of writing to foster self-discovery, creativity, and community. They want to share this gift with others and build a collective of writers supporting one another.

3. What are your fundamental values and principles?
Foster's core values and principles include: creating a nurturing, non-judgmental space for writers, encouraging authentic self-expression, building an interconnected community, facilitating embodied and reflective writing practices, and empowering participants to find their unique voice and vision.

4. What is your vision for the future?
Foster's vision is to cultivate a global community of writers who can access the transformative potential of the written word, unlock their creativity, and use writing as a tool for personal growth, social change, and meaningful connection.

5. Who is your target audience?
Foster's target audience includes writers of all skill levels - from beginners seeking to establish a regular practice to experienced authors looking to deepen their craft and expression. They aim to welcome anyone who wants to explore writing as a form of self-discovery and community building.

6. What specific actions or changes do you want to inspire?
Through their Writing Circles and other offerings, Foster aims to inspire participants to:
- Develop a consistent, embodied writing practice
- Overcome creative blocks and inner critics
- Explore sensitive, vulnerable, or uncharted themes through writing
- Make tangible progress on personal writing projects
- Find their authentic voice and vision

7. What makes your perspective or approach unique?
Foster's unique approach blends community, creativity, and contemplative practices. Their Writing Circles provide a structured, supportive space for writers to connect, write together, and share their work. By incorporating elements like meditation, somatic awareness, and collective questioning, they help writers access deeper layers of self-expression.

8. How do you want to emotionally connect with your audience?
Foster aims to create a genuinely welcoming, non-judgmental, and transformative experience for writers. They want participants to feel seen, witnessed, and supported in this often solitary pursuit. By fostering a sense of belonging, vulnerability, and collective growth, they hope to inspire an emotional connection that empowers writers to take creative risks.

9. What are the key obstacles or challenges you face?
Some key challenges Foster may face include:
- Attracting and retaining a diverse community of writers from different backgrounds and experiences
- Maintaining the integrity of their community-centric approach as they potentially scale
- Helping writers overcome deep-seated fears, doubts, and inner critics around their creative expression
- Ensuring the Writing Circles and other offerings remain accessible and affordable

10. How do you define success for your manifesto?
Foster would define success through metrics such as:
- The number of writers who discover the joy and benefits of a regular writing practice
- The depth of transformation, self-discovery, and creative breakthroughs experienced by their community members
- The quality, vulnerability, and meaningful impact of the written work that emerges from their circles and cohorts
- The strength of the connections, collaborations, and support networks that form within the Foster community

Overall, Foster appears to be a holistic, embodied approach to writing that aims to help people rediscover the transformative power of the written word within a supportive, empowering community context."""


prompt = ChatPromptTemplate.from_template(GUIDELINES_PROMPT)
chain = prompt | model_claude | StrOutputParser()
answer = chain.invoke({})
print(answer)

The Foster Manifesto: Reclaiming the Transformative Power of Writing

I. Introduction

Words have the power to change lives, to reshape reality, to connect souls across time and space. Yet for so many, the act of writing has become a source of anxiety, isolation, and creative blockage rather than a wellspring of joy and transformation. I've felt it myself - the fear of the blank page, the harsh inner critic, the sense that my words don't matter. But through my journey with Foster, I've rediscovered the magic that happens when we come together to write, to share, to witness each other's stories. This manifesto is a call to reclaim writing as a profound tool for personal growth, community building, and social change.

II. Problem Identification

In our hyper-connected yet deeply alienated world, true self-expression has become increasingly rare and precious. We are bombarded by carefully curated social media personas, corporate messaging, and polarized ideologies. The art of writing - of